In [1]:
from pyspark.sql import SparkSession

appName = "Valor total liquido"
master = "local"

# Criand a sessao Spark 
spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .getOrCreate()

22/01/23 20:27:09 WARN Utils: Your hostname, MacBook-Pro-do-Renato-Turtienski.local resolves to a loopback address: 127.0.0.1; using 192.168.0.202 instead (on interface en0)
22/01/23 20:27:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/23 20:27:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#dicionário que será lido como um dataframe para realização das operações necessárias

transacoes = [{'transacao_id':1, 'total_bruto':3000, 'desconto_percentual':6.99},
              {'transacao_id':2, 'total_bruto':57989, 'desconto_percentual':1.45},
              {'transacao_id':4, 'total_bruto':1, 'desconto_percentual':None},
              {'transacao_id':5, 'total_bruto':34, 'desconto_percentual':0.0}]

In [3]:
# criando o dataframe 

df = spark.createDataFrame(transacoes)

In [4]:
# verificando a criacao do dataframe

df.show()
    

+-------------------+-----------+------------+
|desconto_percentual|total_bruto|transacao_id|
+-------------------+-----------+------------+
|               6.99|       3000|           1|
|               1.45|      57989|           2|
|               null|          1|           4|
|                0.0|         34|           5|
+-------------------+-----------+------------+



Será necessário:
1. tratar o percentual, como no primeiro entregável, dividindo os valores do campo desconto_percentual por 100,
2. multiplicar o total bruto por (1-desconto_percentual/100) - lembrando que já estará tratado pela divisão por 100 do primeiro passo
3. Por fim, somar os valores 

In [5]:
''' tratando o campo desconto_percentual, já corrigindo o valor nulo da terceira linha, 
substituindo o por zero, em uma nova coluna para não perdermos informações.
Já traremos o valor que devemos pelo total bruto(passo 2 acima)'''
from pyspark.sql.functions import when   

df = df.withColumn('desconto', when(df.desconto_percentual.isNull(), 1.0).otherwise(1-df.desconto_percentual/100))

In [6]:
df.show()

+-------------------+-----------+------------+--------+
|desconto_percentual|total_bruto|transacao_id|desconto|
+-------------------+-----------+------------+--------+
|               6.99|       3000|           1|  0.9301|
|               1.45|      57989|           2|  0.9855|
|               null|          1|           4|     1.0|
|                0.0|         34|           5|     1.0|
+-------------------+-----------+------------+--------+



In [7]:
''' Para o passo três criaremos mais uma nova coluna (total_liquido) 
com os valores do total_bruto multiplicados pelo desconto ''' 

df = df.withColumn('total_liquido', df.total_bruto*df.desconto)

In [8]:
df.show()

+-------------------+-----------+------------+--------+-------------+
|desconto_percentual|total_bruto|transacao_id|desconto|total_liquido|
+-------------------+-----------+------------+--------+-------------+
|               6.99|       3000|           1|  0.9301|       2790.3|
|               1.45|      57989|           2|  0.9855|   57148.1595|
|               null|          1|           4|     1.0|          1.0|
|                0.0|         34|           5|     1.0|         34.0|
+-------------------+-----------+------------+--------+-------------+



In [9]:
# por fim, somememos os valores da coluna total_liquido, quarta coluna a partir do índice zero

print("{0:.2f}".format(df.groupBy().sum().collect()[0][4]))

59973.46
